## **Dados** sobre o quantitativo de pessoas que terminaram o curso profissionalizante Gestor de Turismo. trata-se de curso gratuito e online, direcionado a atores institucionais públicos e privados, maiores de 18 anos que atuam na gestão do turismo, além de pessoas que tenham interesse em desempenhar atividades relacionadas ao desenvolvimento do setor. Pode ser acessado pelo endereço eletrônico: <p>
http://gestor.turismo.gov.b<p>
Origem do banco de dados.<p>
https://dados.gov.br/dados/conjuntos-dados/gestor-turismo-pessoas-qualificadas<p>
Esse arquivo foi Realizdo uma limpeza e organização dos dados.

##Instale a biblioteca do Google Cloud no Colab


In [ ]:
!pip install --upgrade google-cloud-storage google-cloud-bigquery
!pip install pyspark
!pip install findspark

In [ ]:
import numpy as np
import findspark
import pyspark
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import *
from pyspark.sql.functions import to_date, col, coalesce, date_format, current_date, datediff, floor, when, corr, row_number, lower, upper, avg, stddev, percentile_approx, expr, max, min
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import os
from google.cloud import storage

# Inicialize o cliente
client = storage.Client()

# Defina o nome do bucket
bucket_name = 'soulcode_atividade_05'
bucket = client.bucket(bucket_name)

# Lista de arquivos a serem enviados
arquivos_para_enviar = ['cgt-2018.csv', 'cgt-2019.csv']

# Caminho local onde os arquivos estão armazenados
caminho_local_dos_arquivos = '/content/'  # Substitua pelo caminho real

for nome_arquivo in arquivos_para_enviar:
    caminho_completo_do_arquivo = os.path.join(caminho_local_dos_arquivos, nome_arquivo)

    # Verifique se o arquivo existe
    if os.path.exists(caminho_completo_do_arquivo):
        destination_blob_path = f'Dados_brutos/{nome_arquivo}'
        blob = bucket.blob(destination_blob_path)
        blob.upload_from_filename(caminho_completo_do_arquivo)
        print(f'Arquivo {nome_arquivo} enviado para o Cloud Storage na pasta csv com sucesso!')
    else:
        print(f'Arquivo {nome_arquivo} não encontrado no caminho especificado.')


Arquivo cgt-2018.csv enviado para o Cloud Storage na pasta csv com sucesso!
Arquivo cgt-2019.csv enviado para o Cloud Storage na pasta csv com sucesso!


In [ ]:
# Inicializa o SparkSession
spark = SparkSession.builder \
    .appName("ExemploSpark") \
    .getOrCreate()

# Verifica a versão do Spark para confirmar a configuração
print("Versão do Spark:", spark.version)

Versão do Spark: 3.5.2


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# Torna o pyspark "importável"
findspark.init()

# Crie uma sessão do Spark
spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

# Carregar o DataFrame de um arquivo CSV
dfpk_gestor_turismo_2018 = spark.read.csv("/content/cgt-2018.csv", header=True, inferSchema=True)

dfpk_gestor_turismo_2019 = spark.read.csv("/content/cgt-2019.csv", header=True, inferSchema=True)

dfpk_gestor_turismo_2018.show()
dfpk_gestor_turismo_2019.show()

+-------------+-----------------+--------------+-----------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|Nacionalidade|           Estado|        Cidade|           Bairro|Trabalha|Carteira Assinada|Tipo de Estabelecimento|     Sexo|       Escolaridade|  Raça|Data de Nascimento|Pontuação|Percentual de Conclusão do Canal|Data de Conclusão do Canal|
+-------------+-----------------+--------------+-----------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|       Brasil|Rio Grande do Sul|   São Gabriel|           CENTRO|    NULL|             NULL|              estudante|Masculino|Superior incompleto|Branca|        16-02-1984|      429|                           100.0|      2018-08-24 19:16:...|
|       Brasil|   Rio de

In [ ]:
# Tirar os duplicados
dfpk_gestor_turismo_2018 = dfpk_gestor_turismo_2018.dropDuplicates()
dfpk_gestor_turismo_2019 = dfpk_gestor_turismo_2019.dropDuplicates()

## Arrumando a DataFrame 2018 / 2019

In [ ]:
#retorna colunas com valor null
dfpk_gestor_turismo_2018.select([count(when(col(c).isNull(), c)).alias(c) for c in dfpk_gestor_turismo_2018.columns]).show()

dfpk_gestor_turismo_2019.select([count(when(col(c).isNull(), c)).alias(c) for c in dfpk_gestor_turismo_2019.columns]).show()

+-------------+------+------+------+--------+-----------------+-----------------------+----+------------+----+------------------+---------+--------------------------------+--------------------------+
|Nacionalidade|Estado|Cidade|Bairro|Trabalha|Carteira Assinada|Tipo de Estabelecimento|Sexo|Escolaridade|Raça|Data de Nascimento|Pontuação|Percentual de Conclusão do Canal|Data de Conclusão do Canal|
+-------------+------+------+------+--------+-----------------+-----------------------+----+------------+----+------------------+---------+--------------------------------+--------------------------+
|            0|     3|     3|     3|     824|              824|                      3|   0|           0|   0|                 4|        0|                               0|                         0|
+-------------+------+------+------+--------+-----------------+-----------------------+----+------------+----+------------------+---------+--------------------------------+--------------------------+


In [ ]:
# Filtra o DataFrame para manter apenas as linhas onde a Nacionalidade é "Brasil"
dfpk_gestor_turismo_2018 = dfpk_gestor_turismo_2018.filter(dfpk_gestor_turismo_2018["Nacionalidade"] == "Brasil")
# Exibe as primeiras linhas do DataFrame filtrado para verificar o resultado
dfpk_gestor_turismo_2018.show()

+-------------+-----------------+------------------+----------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|Nacionalidade|           Estado|            Cidade|          Bairro|Trabalha|Carteira Assinada|Tipo de Estabelecimento|     Sexo|       Escolaridade|  Raça|Data de Nascimento|Pontuação|Percentual de Conclusão do Canal|Data de Conclusão do Canal|
+-------------+-----------------+------------------+----------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|       Brasil|         Amazonas|            Manaus|   SANTO ANTÔNIO|    NULL|             NULL|              Estudante| Feminino|Superior incompleto|Branca|        30-05-1989|      429|                           100.0|      2018-12-10 04:09:...|
|       Bras

In [ ]:
# Filtra o DataFrame para manter apenas as linhas onde a Nacionalidade é "Brasil"
dfpk_gestor_turismo_2019 = dfpk_gestor_turismo_2019.filter(dfpk_gestor_turismo_2019["Nacionalidade"] == "Brasil")
# Exibe as primeiras linhas do DataFrame filtrado para verificar o resultado
dfpk_gestor_turismo_2019.show()

+-------------+-----------------+---------------+--------------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|Nacionalidade|           Estado|         Cidade|              Bairro|Trabalha|Carteira Assinada|Tipo de Estabelecimento|     Sexo|       Escolaridade|  Raça|Data de Nascimento|Pontuação|Percentual de Conclusão do Canal|Data de Conclusão do Canal|
+-------------+-----------------+---------------+--------------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|       Brasil|     Minas Gerais|      Arceburgo|              CENTRO|    NULL|             NULL|   Gestor de Turismo...| Feminino|           Superior|Branca|        19-01-1975|      429|                           100.0|      2019-01-26 11:32:...|
|       

In [ ]:
#retorna colunas com valor null
dfpk_gestor_turismo_2018.select([count(when(col(c).isNull(), c)).alias(c) for c in dfpk_gestor_turismo_2018.columns]).show()

dfpk_gestor_turismo_2019.select([count(when(col(c).isNull(), c)).alias(c) for c in dfpk_gestor_turismo_2019.columns]).show()

+-------------+------+------+------+--------+-----------------+-----------------------+----+------------+----+------------------+---------+--------------------------------+--------------------------+
|Nacionalidade|Estado|Cidade|Bairro|Trabalha|Carteira Assinada|Tipo de Estabelecimento|Sexo|Escolaridade|Raça|Data de Nascimento|Pontuação|Percentual de Conclusão do Canal|Data de Conclusão do Canal|
+-------------+------+------+------+--------+-----------------+-----------------------+----+------------+----+------------------+---------+--------------------------------+--------------------------+
|            0|     0|     0|     0|     821|              821|                      0|   0|           0|   0|                 4|        0|                               0|                         0|
+-------------+------+------+------+--------+-----------------+-----------------------+----+------------+----+------------------+---------+--------------------------------+--------------------------+


In [ ]:
# Substituindo todos os valores nulos da coluna "Estado" - "Cidade" - "Bairro" e "Tipo de Estabelecimento"
dfpk_gestor_turismo_2018 = dfpk_gestor_turismo_2018 \
    .fillna('Desconhecido', subset=["Estado"]) \
    .fillna('Desconhecido', subset=["Cidade"]) \
    .fillna('Desconhecido', subset=["Bairro"]) \
    .fillna('Desempregado', subset=["Tipo de Estabelecimento"])

# Substituir valores nulos na coluna 'Data de Nascimento' pela data atual
dfpk_gestor_turismo_2018= dfpk_gestor_turismo_2018.withColumn(
    "Data de Nascimento",
    when(col("Data de Nascimento").isNull(), current_date()).otherwise(col("Data de Nascimento")))

# Alterando erros da coluna 'Bairro'
dateaddf_gestor_turismo_2018 = dfpk_gestor_turismo_2018.withColumn(
    'Bairro',
    when(
        (col('Bairro').rlike('^[0-9]+$')) |
        (col('Bairro') == '-') |
        (col('Bairro').isNull()) |
        (col('Bairro') == '') |
        (col('Bairro') == ' ')
    , 'Desconhecido')
    .otherwise(col('Bairro')))

# Lista de colunas de texto para aplicar a transformação
text_columns = ["Nacionalidade", "Estado", "Cidade", "Bairro", "Trabalha", "Carteira assinada",
                "Tipo de estabelecimento", "Sexo", "Escolaridade", "Raça"]

# Aplicando initcap em todas as colunas de texto
for col_name in text_columns:
    dfpk_gestor_turismo_2018 = dfpk_gestor_turismo_2018.withColumn(col_name, initcap(dfpk_gestor_turismo_2018[col_name]))
dfpk_gestor_turismo_2018.show()

+-------------+-----------------+------------------+----------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|Nacionalidade|           Estado|            Cidade|          Bairro|Trabalha|Carteira assinada|Tipo de estabelecimento|     Sexo|       Escolaridade|  Raça|Data de Nascimento|Pontuação|Percentual de Conclusão do Canal|Data de Conclusão do Canal|
+-------------+-----------------+------------------+----------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|       Brasil|         Amazonas|            Manaus|   Santo Antônio|    NULL|             NULL|              Estudante| Feminino|Superior Incompleto|Branca|        30-05-1989|      429|                           100.0|      2018-12-10 04:09:...|
|       Bras

In [ ]:
# Substituindo todos os valores nulos da coluna "Estado" - "Cidade" - "Bairro" e "Tipo de Estabelecimento"
dfpk_gestor_turismo_2019 = dfpk_gestor_turismo_2019 \
    .fillna('Desconhecido', subset=["Estado"]) \
    .fillna('Desconhecido', subset=["Cidade"]) \
    .fillna('Desconhecido', subset=["Bairro"]) \
    .fillna('Desempregado', subset=["Tipo de Estabelecimento"])

# Substituir valores nulos na coluna 'Data de Nascimento' pela data atual
dfpk_gestor_turismo_2019= dfpk_gestor_turismo_2019.withColumn(
    "Data de Nascimento",
    when(col("Data de Nascimento").isNull(), current_date()).otherwise(col("Data de Nascimento")))

# Alterando erros da coluna 'Bairro'
dateaddf_gestor_turismo_2019 = dfpk_gestor_turismo_2019.withColumn(
    'Bairro',
    when(
        (col('Bairro').rlike('^[0-9]+$')) |
        (col('Bairro') == '-') |
        (col('Bairro').isNull()) |
        (col('Bairro') == '') |
        (col('Bairro') == ' ')
    , 'Desconhecido')
    .otherwise(col('Bairro')))

# Lista de colunas de texto para aplicar a transformação
text_columns = ["Nacionalidade", "Estado", "Cidade", "Bairro", "Trabalha", "Carteira assinada",
                "Tipo de estabelecimento", "Sexo", "Escolaridade", "Raça"]

# Aplicando initcap em todas as colunas de texto
for col_name in text_columns:
    dfpk_gestor_turismo_2019 = dfpk_gestor_turismo_2019.withColumn(col_name, initcap(dfpk_gestor_turismo_2019[col_name]))
dfpk_gestor_turismo_2019.show()

+-------------+-----------------+---------------+--------------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|Nacionalidade|           Estado|         Cidade|              Bairro|Trabalha|Carteira assinada|Tipo de estabelecimento|     Sexo|       Escolaridade|  Raça|Data de Nascimento|Pontuação|Percentual de Conclusão do Canal|Data de Conclusão do Canal|
+-------------+-----------------+---------------+--------------------+--------+-----------------+-----------------------+---------+-------------------+------+------------------+---------+--------------------------------+--------------------------+
|       Brasil|     Minas Gerais|      Arceburgo|              Centro|    NULL|             NULL|   Gestor De Turismo...| Feminino|           Superior|Branca|        19-01-1975|      429|                           100.0|      2019-01-26 11:32:...|
|       

In [ ]:

# Supondo que você já tenha um DataFrame PySpark chamado df
dfpd_gestor_turismo_2018 = dfpk_gestor_turismo_2018.toPandas()

# Agora df_pandas é um DataFrame do pandas e você pode trabalhar com ele usando a API do pandas
print(dfpd_gestor_turismo_2018.head())

  Nacionalidade          Estado          Cidade            Bairro Trabalha  \
0        Brasil        Amazonas          Manaus     Santo Antônio     None   
1        Brasil  Rio De Janeiro        Quissamã  Canto Da Saudade     None   
2        Brasil           Ceará       Fortaleza        Vila Velha     None   
3        Brasil  Rio De Janeiro  Rio De Janeiro      Brás De Pina     None   
4        Brasil      Pernambuco          Recife        Parnamirin     None   

  Carteira assinada    Tipo de estabelecimento       Sexo  \
0              None                  Estudante   Feminino   
1              None  Gestor De Turismo Público  Masculino   
2              None                  Estudante   Feminino   
3              None            Guia De Turismo  Masculino   
4              None  Gestor De Turismo Público   Feminino   

          Escolaridade    Raça Data de Nascimento  Pontuação  \
0  Superior Incompleto  Branca         30-05-1989        429   
1                Medio  Branca      

In [ ]:

# Supondo que você já tenha um DataFrame PySpark chamado df
dfpd_gestor_turismo_2019 = dfpk_gestor_turismo_2019.toPandas()

# Agora df_pandas é um DataFrame do pandas e você pode trabalhar com ele usando a API do pandas
print(dfpd_gestor_turismo_2019.head())

  Nacionalidade             Estado          Cidade          Bairro Trabalha  \
0        Brasil       Minas Gerais       Arceburgo          Centro     None   
1        Brasil          São Paulo   Fernandópolis  Jardim Paraiso     None   
2        Brasil     Rio De Janeiro  Rio De Janeiro      Santa Cruz     None   
3        Brasil          São Paulo       São Paulo        Perdizes     None   
4        Brasil  Rio Grande Do Sul         Colinas        Interior     None   

  Carteira assinada    Tipo de estabelecimento       Sexo Escolaridade  \
0              None  Gestor De Turismo Privado   Feminino     Superior   
1              None            Guia De Turismo   Feminino        Medio   
2              None            Guia De Turismo  Masculino        Medio   
3              None  Gestor De Turismo Privado   Feminino     Superior   
4              None           Simples Nacional  Masculino     Superior   

     Raça Data de Nascimento  Pontuação  Percentual de Conclusão do Canal  \
0  

In [ ]:
display(dfpd_gestor_turismo_2018)

,Nacionalidade,Estado,Cidade,Bairro,Trabalha,Carteira assinada,Tipo de estabelecimento,Sexo,Escolaridade,Raça,Data de Nascimento,Pontuação,Percentual de Conclusão do Canal,Data de Conclusão do Canal
0,Brasil,Amazonas,Manaus,Santo Antônio,None,None,Estudante,Feminino,Superior Incompleto,Branca,30-05-1989,429,100.0,2018-12-10 04:09:00.668
1,Brasil,Rio De Janeiro,Quissamã,Canto Da Saudade,None,None,Gestor De Turismo Público,Masculino,Medio,Branca,19-03-1965,429,100.0,2018-09-14 23:12:58.523
2,Brasil,Ceará,Fortaleza,Vila Velha,None,None,Estudante,Feminino,Superior,Parda,14-04-1989,429,100.0,2018-10-25 21:18:04.246
3,Brasil,Rio De Janeiro,Rio De Janeiro,Brás De Pina,None,None,Guia De Turismo,Masculino,Superior,Parda,02-07-1982,429,100.0,2018-12-01 19:01:00.411
4,Brasil,Pernambuco,Recife,Parnamirin,None,None,Gestor De Turismo Público,Feminino,Superior Incompleto,Branca,06-08-1959,429,100.0,2018-11-04 01:33:41.822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,Brasil,Pernambuco,Cupira,Centro,None,None,Gestor De Turismo Público,Feminino,Superior,Branca,30-06-1977,429,100.0,2018-11-15 01:51:17.550
817,Brasil,Maranhão,São Luís,Bequimão,None,None,Gestor De Turismo Público,Feminino,Superior Incompleto,Parda,21-03-2000,429,100.0,2018-10-01 01:49:38.797
818,Brasil,Rondônia,Porto Velho,Cohab Floresta,None,None,Estudante De Gestão,Masculino,Superior,Parda,02-02-1997,429,100.0,2018-11-07 14:07:04.077
819,Brasil,Rio Grande Do Sul,São Borja,Betin,None,None,Gestão De Turismo,Masculino,Superior Incompleto,Parda,24-11-1980,429,100.0,2018-09-13 02:47:21.484


In [ ]:
display(dfpd_gestor_turismo_2019)

,Nacionalidade,Estado,Cidade,Bairro,Trabalha,Carteira assinada,Tipo de estabelecimento,Sexo,Escolaridade,Raça,Data de Nascimento,Pontuação,Percentual de Conclusão do Canal,Data de Conclusão do Canal
0,Brasil,Minas Gerais,Arceburgo,Centro,None,None,Gestor De Turismo Privado,Feminino,Superior,Branca,19-01-1975,429,100.0,2019-01-26 11:32:28.924
1,Brasil,São Paulo,Fernandópolis,Jardim Paraiso,None,None,Guia De Turismo,Feminino,Medio,Branca,21-01-1968,429,100.0,2019-02-16 23:22:55.398
2,Brasil,Rio De Janeiro,Rio De Janeiro,Santa Cruz,None,None,Guia De Turismo,Masculino,Medio,Parda,18-12-1965,429,100.0,2019-11-16 00:04:14.814
3,Brasil,São Paulo,São Paulo,Perdizes,None,None,Gestor De Turismo Privado,Feminino,Superior,Branca,11-03-1957,429,100.0,2019-04-30 19:48:08.819
4,Brasil,Rio Grande Do Sul,Colinas,Interior,None,None,Simples Nacional,Masculino,Superior,Branca,20-05-1973,429,100.0,2019-11-24 23:07:03.475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,Brasil,São Paulo,Iacanga,Jardim Brasil,None,None,Gestor De Turismo Público,Feminino,Superior,Branca,07-08-1978,429,100.0,2019-12-23 15:54:12.235
1422,Brasil,Mato Grosso Do Sul,Sonora,Centro,None,None,Gestor De Turismo Público,Masculino,Superior,Parda,08-05-1959,429,100.0,2019-02-08 17:02:50.229
1423,Brasil,Sergipe,Boquim,Zona Rural,None,None,Monitor De Transport,Masculino,Medio Incompleto,Parda,26-11-1995,429,100.0,2019-10-30 11:46:30.728
1424,Brasil,Maranhão,São Luís,Cohab,None,None,Gestor De Turismo Privado,Masculino,Superior,Parda,15-03-1990,429,100.0,2019-02-17 14:30:39.823


In [ ]:
# 1. Normalizar valores: transformar primeira letra em maiúscula e remover espaços extras
dfpd_gestor_turismo_2018["Tipo de estabelecimento"] = dfpd_gestor_turismo_2018["Tipo de estabelecimento"].str.strip().str.capitalize()

# 2. Substituir valores não esperados por 'Outros'
dfpd_gestor_turismo_2018["Tipo de estabelecimento"] = dfpd_gestor_turismo_2018["Tipo de estabelecimento"].replace(['-', ''], 'Outros')

# Validar se as mudanças foram aplicadas corretamente
print(dfpd_gestor_turismo_2018["Tipo de estabelecimento"].unique())

# Exibir as primeiras linhas do DataFrame para verificar
print(dfpd_gestor_turismo_2018.head())


['Estudante' 'Gestor de turismo público' 'Guia de turismo'
 'Gestor de turismo privado' 'Rep. medicamentos' 'Estudante turismo'
 'Educação' 'Desempregada' 'Funcionário público' 'Bancario'
 'Guia turístico' 'Professor universitá' 'Estudante de turismo'
 'Empresario' 'Agente de viagem' 'Aux. administrativo'
 'Assessor de turismo' 'Hotelaria hospitalar' 'Agente de turismo'
 'Gestor público' 'Comércio exterior' 'Turismo urbano' 'Ong'
 'Estudante de graduaç' 'Militar reformado' 'Turismóloga' 'Docente'
 'Condutora de turismo' 'Estagiário casa do t' 'Turismo'
 'Produtora de eventos' 'Técnico enfermagem' 'Secretária' 'Professor'
 'Aeroviaria' 'Entidade/associação' 'Turismologa' 'Professor de turismo'
 'Técnico adm.' 'Coordenadora de viag' 'Autonomo' 'Administração'
 'Cultura' 'Hospedagem' 'Letras' 'Gestão em turismo' 'Almoxarife'
 'Consorcio de turismo' 'Educador cultural' 'Consultor'
 'Turismo e eventos' 'Licenciado em turism' 'Bacharel em turismo'
 'Diretora de projetos' 'Agente de viagens' 

In [ ]:
# 1. Normalizar valores: transformar primeira letra em maiúscula e remover espaços extras
dfpd_gestor_turismo_2019["Tipo de estabelecimento"] = dfpd_gestor_turismo_2018["Tipo de estabelecimento"].str.strip().str.capitalize()

# 2. Substituir valores não esperados por 'Outros'
dfpd_gestor_turismo_2019["Tipo de estabelecimento"] = dfpd_gestor_turismo_2018["Tipo de estabelecimento"].replace(['-', ''], 'Outros')

# Validar se as mudanças foram aplicadas corretamente
print(dfpd_gestor_turismo_2019["Tipo de estabelecimento"].unique())

# Exibir as primeiras linhas do DataFrame para verificar
print(dfpd_gestor_turismo_2019.head())

['Estudante' 'Gestor de turismo público' 'Guia de turismo'
 'Gestor de turismo privado' 'Rep. medicamentos' 'Estudante turismo'
 'Educação' 'Desempregada' 'Funcionário público' 'Bancario'
 'Guia turístico' 'Professor universitá' 'Estudante de turismo'
 'Empresario' 'Agente de viagem' 'Aux. administrativo'
 'Assessor de turismo' 'Hotelaria hospitalar' 'Agente de turismo'
 'Gestor público' 'Comércio exterior' 'Turismo urbano' 'Ong'
 'Estudante de graduaç' 'Militar reformado' 'Turismóloga' 'Docente'
 'Condutora de turismo' 'Estagiário casa do t' 'Turismo'
 'Produtora de eventos' 'Técnico enfermagem' 'Secretária' 'Professor'
 'Aeroviaria' 'Entidade/associação' 'Turismologa' 'Professor de turismo'
 'Técnico adm.' 'Coordenadora de viag' 'Autonomo' 'Administração'
 'Cultura' 'Hospedagem' 'Letras' 'Gestão em turismo' 'Almoxarife'
 'Consorcio de turismo' 'Educador cultural' 'Consultor'
 'Turismo e eventos' 'Licenciado em turism' 'Bacharel em turismo'
 'Diretora de projetos' 'Agente de viagens' 

In [ ]:
import numpy as np

# Definir as condições para a coluna "Trabalha"
conditions_trabalha = [
    (dfpd_gestor_turismo_2018["Tipo de estabelecimento"] == "Estudante"),
    (dfpd_gestor_turismo_2018["Tipo de estabelecimento"] == "Gestor de turismo público")
]

# Definir os valores a serem atribuídos para cada condição
choices_trabalha = ["Não", "Sim"]

# Aplicar a lógica condicional para "Trabalha"
dfpd_gestor_turismo_2018["Trabalha"] = np.select(conditions_trabalha, choices_trabalha, default="Sim")

# Definir as condições para a coluna "Carteira Assinada"
conditions_carteira = [
    (dfpd_gestor_turismo_2018["Tipo de estabelecimento"] == "Gestor de turismo público") &
    (dfpd_gestor_turismo_2018["Escolaridade"] == "Superior"),

    (dfpd_gestor_turismo_2018["Tipo de estabelecimento"] == "Turismólogo") &
    (dfpd_gestor_turismo_2018["Escolaridade"] == "Superior")
]

# Definir os valores a serem atribuídos para cada condição
choices_carteira = ["Sim", "Sim"]

# Aplicar a lógica condicional para "Carteira Assinada"
dfpd_gestor_turismo_2018["Carteira Assinada"] = np.select(conditions_carteira, choices_carteira, default="Não")

# Exibir o DataFrame atualizado
display(dfpd_gestor_turismo_2018.head())


,Nacionalidade,Estado,Cidade,Bairro,Trabalha,Carteira assinada,Tipo de estabelecimento,Sexo,Escolaridade,Raça,Data de Nascimento,Pontuação,Percentual de Conclusão do Canal,Data de Conclusão do Canal,Carteira Assinada
0,Brasil,Amazonas,Manaus,Santo Antônio,Não,None,Estudante,Feminino,Superior Incompleto,Branca,30-05-1989,429,100.0,2018-12-10 04:09:00.668,Não
1,Brasil,Rio De Janeiro,Quissamã,Canto Da Saudade,Sim,None,Gestor de turismo público,Masculino,Medio,Branca,19-03-1965,429,100.0,2018-09-14 23:12:58.523,Não
2,Brasil,Ceará,Fortaleza,Vila Velha,Não,None,Estudante,Feminino,Superior,Parda,14-04-1989,429,100.0,2018-10-25 21:18:04.246,Não
3,Brasil,Rio De Janeiro,Rio De Janeiro,Brás De Pina,Sim,None,Guia de turismo,Masculino,Superior,Parda,02-07-1982,429,100.0,2018-12-01 19:01:00.411,Não
4,Brasil,Pernambuco,Recife,Parnamirin,Sim,None,Gestor de turismo público,Feminino,Superior Incompleto,Branca,06-08-1959,429,100.0,2018-11-04 01:33:41.822,Não


In [ ]:
# Definir as condições para a coluna "Trabalha"
conditions_trabalha = [
    (dfpd_gestor_turismo_2019["Tipo de estabelecimento"] == "estudante"),
    (dfpd_gestor_turismo_2019["Tipo de estabelecimento"] == "Gestor de Turismo Público")
]

# Definir os valores a serem atribuídos para cada condição
choices_trabalha = ["Não", "Sim"]

# Aplicar a lógica condicional para "Trabalha"
dfpd_gestor_turismo_2019["Trabalha"] = np.select(conditions_trabalha, choices_trabalha, default="Sim")

# Definir as condições para a coluna "Carteira Assinada"
conditions_carteira = [
    (dfpd_gestor_turismo_2019["Tipo de estabelecimento"] == "Gestor de Turismo Público") &
    (dfpd_gestor_turismo_2019["Escolaridade"] == "Superior"),

    (dfpd_gestor_turismo_2019["Tipo de estabelecimento"] == "TURISMÓLOGO") &
    (dfpd_gestor_turismo_2019["Escolaridade"] == "Superior")
]

# Definir os valores a serem atribuídos para cada condição
choices_carteira = ["Sim", "Sim"]

# Aplicar a lógica condicional para "Carteira Assinada"
dfpd_gestor_turismo_2019["Carteira Assinada"] = np.select(conditions_carteira, choices_carteira, default="Não")

# Exibir o DataFrame atualizado
display(dfpd_gestor_turismo_2019)

,Nacionalidade,Estado,Cidade,Bairro,Trabalha,Carteira assinada,Tipo de estabelecimento,Sexo,Escolaridade,Raça,Data de Nascimento,Pontuação,Percentual de Conclusão do Canal,Data de Conclusão do Canal,Carteira Assinada
0,Brasil,Minas Gerais,Arceburgo,Centro,Sim,None,Estudante,Feminino,Superior,Branca,19-01-1975,429,100.0,2019-01-26 11:32:28.924,Não
1,Brasil,São Paulo,Fernandópolis,Jardim Paraiso,Sim,None,Gestor de turismo público,Feminino,Medio,Branca,21-01-1968,429,100.0,2019-02-16 23:22:55.398,Não
2,Brasil,Rio De Janeiro,Rio De Janeiro,Santa Cruz,Sim,None,Estudante,Masculino,Medio,Parda,18-12-1965,429,100.0,2019-11-16 00:04:14.814,Não
3,Brasil,São Paulo,São Paulo,Perdizes,Sim,None,Guia de turismo,Feminino,Superior,Branca,11-03-1957,429,100.0,2019-04-30 19:48:08.819,Não
4,Brasil,Rio Grande Do Sul,Colinas,Interior,Sim,None,Gestor de turismo público,Masculino,Superior,Branca,20-05-1973,429,100.0,2019-11-24 23:07:03.475,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,Brasil,São Paulo,Iacanga,Jardim Brasil,Sim,None,NaN,Feminino,Superior,Branca,07-08-1978,429,100.0,2019-12-23 15:54:12.235,Não
1422,Brasil,Mato Grosso Do Sul,Sonora,Centro,Sim,None,NaN,Masculino,Superior,Parda,08-05-1959,429,100.0,2019-02-08 17:02:50.229,Não
1423,Brasil,Sergipe,Boquim,Zona Rural,Sim,None,NaN,Masculino,Medio Incompleto,Parda,26-11-1995,429,100.0,2019-10-30 11:46:30.728,Não
1424,Brasil,Maranhão,São Luís,Cohab,Sim,None,NaN,Masculino,Superior,Parda,15-03-1990,429,100.0,2019-02-17 14:30:39.823,Não


In [ ]:
dfpd_gestor_turismo_2018.drop('Carteira assinada', axis=1, inplace=True)

display(dfpd_gestor_turismo_2018)

,Nacionalidade,Estado,Cidade,Bairro,Trabalha,Tipo de estabelecimento,Sexo,Escolaridade,Raça,Data de Nascimento,Pontuação,Percentual de Conclusão do Canal,Data de Conclusão do Canal,Carteira Assinada
0,Brasil,Amazonas,Manaus,Santo Antônio,Não,Estudante,Feminino,Superior Incompleto,Branca,30-05-1989,429,100.0,2018-12-10 04:09:00.668,Não
1,Brasil,Rio De Janeiro,Quissamã,Canto Da Saudade,Sim,Gestor de turismo público,Masculino,Medio,Branca,19-03-1965,429,100.0,2018-09-14 23:12:58.523,Não
2,Brasil,Ceará,Fortaleza,Vila Velha,Não,Estudante,Feminino,Superior,Parda,14-04-1989,429,100.0,2018-10-25 21:18:04.246,Não
3,Brasil,Rio De Janeiro,Rio De Janeiro,Brás De Pina,Sim,Guia de turismo,Masculino,Superior,Parda,02-07-1982,429,100.0,2018-12-01 19:01:00.411,Não
4,Brasil,Pernambuco,Recife,Parnamirin,Sim,Gestor de turismo público,Feminino,Superior Incompleto,Branca,06-08-1959,429,100.0,2018-11-04 01:33:41.822,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,Brasil,Pernambuco,Cupira,Centro,Sim,Gestor de turismo público,Feminino,Superior,Branca,30-06-1977,429,100.0,2018-11-15 01:51:17.550,Sim
817,Brasil,Maranhão,São Luís,Bequimão,Sim,Gestor de turismo público,Feminino,Superior Incompleto,Parda,21-03-2000,429,100.0,2018-10-01 01:49:38.797,Não
818,Brasil,Rondônia,Porto Velho,Cohab Floresta,Sim,Estudante de gestão,Masculino,Superior,Parda,02-02-1997,429,100.0,2018-11-07 14:07:04.077,Não
819,Brasil,Rio Grande Do Sul,São Borja,Betin,Sim,Gestão de turismo,Masculino,Superior Incompleto,Parda,24-11-1980,429,100.0,2018-09-13 02:47:21.484,Não


In [ ]:
dfpd_gestor_turismo_2019.drop('Carteira assinada', axis=1, inplace=True)

display(dfpd_gestor_turismo_2019)

,Nacionalidade,Estado,Cidade,Bairro,Trabalha,Tipo de estabelecimento,Sexo,Escolaridade,Raça,Data de Nascimento,Pontuação,Percentual de Conclusão do Canal,Data de Conclusão do Canal,Carteira Assinada
0,Brasil,Minas Gerais,Arceburgo,Centro,Sim,Estudante,Feminino,Superior,Branca,19-01-1975,429,100.0,2019-01-26 11:32:28.924,Não
1,Brasil,São Paulo,Fernandópolis,Jardim Paraiso,Sim,Gestor de turismo público,Feminino,Medio,Branca,21-01-1968,429,100.0,2019-02-16 23:22:55.398,Não
2,Brasil,Rio De Janeiro,Rio De Janeiro,Santa Cruz,Sim,Estudante,Masculino,Medio,Parda,18-12-1965,429,100.0,2019-11-16 00:04:14.814,Não
3,Brasil,São Paulo,São Paulo,Perdizes,Sim,Guia de turismo,Feminino,Superior,Branca,11-03-1957,429,100.0,2019-04-30 19:48:08.819,Não
4,Brasil,Rio Grande Do Sul,Colinas,Interior,Sim,Gestor de turismo público,Masculino,Superior,Branca,20-05-1973,429,100.0,2019-11-24 23:07:03.475,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,Brasil,São Paulo,Iacanga,Jardim Brasil,Sim,NaN,Feminino,Superior,Branca,07-08-1978,429,100.0,2019-12-23 15:54:12.235,Não
1422,Brasil,Mato Grosso Do Sul,Sonora,Centro,Sim,NaN,Masculino,Superior,Parda,08-05-1959,429,100.0,2019-02-08 17:02:50.229,Não
1423,Brasil,Sergipe,Boquim,Zona Rural,Sim,NaN,Masculino,Medio Incompleto,Parda,26-11-1995,429,100.0,2019-10-30 11:46:30.728,Não
1424,Brasil,Maranhão,São Luís,Cohab,Sim,NaN,Masculino,Superior,Parda,15-03-1990,429,100.0,2019-02-17 14:30:39.823,Não


In [ ]:
# Adiciona a coluna 'Ano' a cada DataFrame
dfpd_gestor_turismo_2018['Ano'] = 2018
dfpd_gestor_turismo_2019['Ano'] = 2019

# Junta os DataFrames
dfpd_gestor_turismo_total = pd.concat([dfpd_gestor_turismo_2018, dfpd_gestor_turismo_2019], ignore_index=True)

# Exibe o DataFrame resultante
display(dfpd_gestor_turismo_total)


,Nacionalidade,Estado,Cidade,Bairro,Trabalha,Tipo de estabelecimento,Sexo,Escolaridade,Raça,Data de Nascimento,Pontuação,Percentual de Conclusão do Canal,Data de Conclusão do Canal,Carteira Assinada,Ano
0,Brasil,Amazonas,Manaus,Santo Antônio,Não,Estudante,Feminino,Superior Incompleto,Branca,30-05-1989,429,100.0,2018-12-10 04:09:00.668,Não,2018
1,Brasil,Rio De Janeiro,Quissamã,Canto Da Saudade,Sim,Gestor de turismo público,Masculino,Medio,Branca,19-03-1965,429,100.0,2018-09-14 23:12:58.523,Não,2018
2,Brasil,Ceará,Fortaleza,Vila Velha,Não,Estudante,Feminino,Superior,Parda,14-04-1989,429,100.0,2018-10-25 21:18:04.246,Não,2018
3,Brasil,Rio De Janeiro,Rio De Janeiro,Brás De Pina,Sim,Guia de turismo,Masculino,Superior,Parda,02-07-1982,429,100.0,2018-12-01 19:01:00.411,Não,2018
4,Brasil,Pernambuco,Recife,Parnamirin,Sim,Gestor de turismo público,Feminino,Superior Incompleto,Branca,06-08-1959,429,100.0,2018-11-04 01:33:41.822,Não,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2242,Brasil,São Paulo,Iacanga,Jardim Brasil,Sim,NaN,Feminino,Superior,Branca,07-08-1978,429,100.0,2019-12-23 15:54:12.235,Não,2019
2243,Brasil,Mato Grosso Do Sul,Sonora,Centro,Sim,NaN,Masculino,Superior,Parda,08-05-1959,429,100.0,2019-02-08 17:02:50.229,Não,2019
2244,Brasil,Sergipe,Boquim,Zona Rural,Sim,NaN,Masculino,Medio Incompleto,Parda,26-11-1995,429,100.0,2019-10-30 11:46:30.728,Não,2019
2245,Brasil,Maranhão,São Luís,Cohab,Sim,NaN,Masculino,Superior,Parda,15-03-1990,429,100.0,2019-02-17 14:30:39.823,Não,2019


## Salvando e enviando para o Bucket

In [ ]:
# Salvar DataFrame como CSV localmente
dfpd_gestor_turismo_2018.to_csv('gestor_turismo_2018_atualizado.csv', index=False)
dfpd_gestor_turismo_2019.to_csv('gestor_turismo_2019_atualizado.csv', index=False)
dfpd_gestor_turismo_total.to_csv('dfpd_gestor_turismo_total.csv', index=False)

In [ ]:
from google.cloud import storage

# Configurações do bucket e caminho do arquivo
bucket_name = 'soulcode_atividade_05'
folder_name = 'Dados_limpos/'  # Caminho da pasta dentro do bucket
file_path_2018 = 'gestor_turismo_2018_atualizado.csv'
file_path_2019 = 'gestor_turismo_2019_atualizado.csv'
file_path_2018_2019 = 'dfpd_gestor_turismo_total.csv'
# Criar o cliente de armazenamento
client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Função para fazer upload do arquivo para o bucket
def upload_to_bucket(local_file, bucket, blob_name):
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(local_file)
    print(f'Arquivo {local_file} enviado para o bucket {bucket_name} como {blob_name}')

# Fazer upload dos arquivos CSV para a pasta 'Dados_limpos' no bucket
upload_to_bucket(file_path_2018, bucket, folder_name + file_path_2018)
upload_to_bucket(file_path_2019, bucket, folder_name + file_path_2019)
upload_to_bucket(file_path_2018_2019, bucket, folder_name + file_path_2018_2019)

Arquivo gestor_turismo_2018_atualizado.csv enviado para o bucket soulcode_atividade_05 como Dados_limpos/gestor_turismo_2018_atualizado.csv
Arquivo gestor_turismo_2019_atualizado.csv enviado para o bucket soulcode_atividade_05 como Dados_limpos/gestor_turismo_2019_atualizado.csv
Arquivo dfpd_gestor_turismo_total.csv enviado para o bucket soulcode_atividade_05 como Dados_limpos/dfpd_gestor_turismo_total.csv


## Insights = Percepções

In [ ]:
# Quantas pessoas qualificadas dentro de cada estado nos anos 2018 e 2019
tabela_Estado = dfpd_gestor_turismo_total.groupby(['Estado', 'Ano']).size().unstack(fill_value=0)

# Exemplo de como selecionar um estado específico (por exemplo, 'SP')
estado_selecionado = 'São Paulo'
resultado = tabela_Estado.loc[estado_selecionado]

# Exibe o resultado
print(resultado)

tabela_Estado.to_csv('tabela_Estado.csv', index=False)

Ano
2018    136
2019    314
Name: São Paulo, dtype: int64


In [ ]:
# Quantas pessoas qualificadas dentro de cada cidade nos anos 2018 e 2019
tabela_Cidade = dfpd_gestor_turismo_total.groupby(['Estado','Cidade', 'Ano']).size().unstack(fill_value=0)
# Exemplo de como selecionar um estado específico (por exemplo, 'SP')
estado_selecionado = 'Bahia'
resultado = tabela_Cidade.loc[estado_selecionado]

# Exibe o resultado
print(resultado)

tabela_Cidade.to_csv('tabela_Cidade.csv', index=False)

Ano                     2018  2019
Cidade                            
Abaíra                     1     1
Alagoinhas                 1     0
Amargosa                   1     0
Barra Do Mendes            0     1
Barreiras                  1     1
Bom Jesus Da Lapa          0     1
Bonito                     1     0
Cairu                      3     0
Camaçari                   0     4
Candeias                   0     1
Casa Nova                  0     1
Dom Macedo Costa           1     0
Entre Rios                 1     0
Eunápolis                  4     2
Feira De Santana           0     2
Guaratinga                 1     1
Ibicaraí                   0     1
Ilhéus                     2     4
Ipiaú                      1     1
Itacaré                    0     1
Itamaraju                  0     1
Itaparica                  0     1
Ituberá                    0     1
Jaborandi                  0     1
Jacobina                   1     0
Jandaíra                   1     0
Jequié              

In [ ]:
# vendo referencia de grau de escolaridade referenciado pelo "Tipo de estabelecimento"
tabela_estabelecimento = dfpd_gestor_turismo_total.groupby(['Tipo de estabelecimento', 'Escolaridade']).size().unstack(fill_value=0)
display(tabela_estabelecimento)

tabela_estabelecimento.to_csv('tabela_estabelecimento.csv', index=False)

Escolaridade,Fundamental,Medio,Medio Incompleto,Superior,Superior Incompleto
Tipo de estabelecimento,,,,,
Acadêmico,0,1,0,0,1
Acadêmico de turismo,0,0,0,1,1
Acs,0,1,0,1,0
Administrador,0,0,0,2,2
Administradora,0,0,0,2,0
...,...,...,...,...,...
Venderora,0,1,0,1,0
Vendora,0,1,0,1,0
Viajante,0,0,0,0,2


## Enviando Insight para o Bucket


In [ ]:
from google.cloud import storage
tabela_Estado_path = 'tabela_Estado.csv'
tabela_Cidade_path = 'tabela_Cidade.csv'
tabela_estabelecimento_path = 'tabela_estabelecimento.csv'

# Configurações do bucket e caminho do arquivo
bucket_name = 'soulcode_atividade_05'
folder_name = 'insights/'

# Criar o cliente de armazenamento
client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Função para fazer upload do arquivo para o bucket
def upload_to_bucket(local_file, bucket, blob_name):
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(local_file)
    print(f'Arquivo {local_file} enviado para o bucket {bucket_name} como {blob_name}')

# Fazer upload dos arquivos CSV para a pasta 'Dados_limpos' no bucket
upload_to_bucket(tabela_Estado_path, bucket, folder_name + 'tabela_Estado.csv')
upload_to_bucket(tabela_Cidade_path, bucket, folder_name + 'tabela_Cidade.csv')
upload_to_bucket(tabela_estabelecimento_path, bucket, folder_name + 'tabela_estabelecimento.csv')


Arquivo tabela_Estado.csv enviado para o bucket soulcode_atividade_05 como insights/tabela_Estado.csv
Arquivo tabela_Cidade.csv enviado para o bucket soulcode_atividade_05 como insights/tabela_Cidade.csv
Arquivo tabela_estabelecimento.csv enviado para o bucket soulcode_atividade_05 como insights/tabela_estabelecimento.csv
